In [1]:
import os, sys
project_dir = 'C:/Users/zyt/Documents/GitHub Repositories/codeclf_gui/codeclf'
sys.path.append(project_dir)

In [2]:
from utils.CodeTokenizer import CodeTokenizer, CodeSplitTokenizer
from utils.CodeTokenizer import ContextCodeTokenizer, ContextCodeSplitTokenizer
from utils.Utils import timethis
from preprocessing.DataProcessor import DataProcessor
from train.ClfModel import ContextModel

import numpy as np
import pandas as pd
import logging
import tensorflow as tf

In [3]:
def feature_to_id_bta(features, label):
    return ccst.from_feature_to_token_id_bta(features[0].decode("utf-8"),
                                            features[1].decode("utf-8"),
                                            features[2].decode("utf-8")), label

In [4]:
def tf_feature_to_id_bta(features, label):
    label_shape = label.shape
    [features, label] = tf.numpy_function(feature_to_id_bta,
                                inp=[features, label],
                                Tout=[tf.int32, tf.int64])
    features.set_shape((60,))
    label.set_shape(label_shape)
    return features, label

In [5]:
# 未平衡两类别
ccst = ContextCodeSplitTokenizer(os.path.join(project_dir, 'vocabs/split_simple_vocab50000.txt'))
dp = DataProcessor()

corpus_path = '../datasets/df_test_corpus.tar.bz2'
df_data = pd.read_pickle(corpus_path)
data = df_data['code']
ds = dp.process_context_tfdata_merge(data)  # 自动标注

ds


df_codes:330684, df_docs:192157


<TensorSliceDataset shapes: ((3,), ()), types: (tf.string, tf.int64)>

In [6]:
ds_test = ds.map(tf_feature_to_id_bta)
ds_test

<MapDataset shapes: ((60,), ()), types: (tf.int32, tf.int64)>

In [7]:
# 平衡两类别
test_code_ds = (ds_test
    .filter(lambda features, label: label == 0).shuffle(100000).repeat())
test_docs_ds = (ds_test
    .filter(lambda features, label: label == 1).shuffle(100000).repeat())
ds_test = tf.data.experimental.sample_from_datasets(
            [test_code_ds, test_docs_ds],
            weights=[0.5, 0.5])
# ds_test = ds_test.shuffle(10000).batch(32).prefetch(2)
ds_test = ds_test.batch(1024).prefetch(2)

Cause: could not parse the source code:

    .filter(lambda features, label: label == 0).shuffle(100000).repeat())

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: could not parse the source code:

    .filter(lambda features, label: label == 0).shuffle(100000).repeat())

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: could not parse the source code:

    .filter(lambda features, label: label == 0).shuffle(100000).repeat())

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code:

    .filter(lambda features, label: label == 1).shuffle(100000).repeat())

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: could not parse the source code:

    .filter(lambda features, label: label == 1).shuffle(100000).repeat())

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: could not parse the source code:

    .filter(lambda features, label: label == 1).shuffle(100000).repeat())

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [31]:
count_code = 0
count_docs = 0
for f, l in ds_test.take(1):
    # print(f, l)
    for label in l:
        if label == 0:
            count_code += 1
        else:
            count_docs += 1
print(count_code, count_docs)

13 19


In [ ]:
%%time
for f, l in ds_test.take(10):
    # print(f, l)
    print(l.numpy().mean())